Introduction 

In [10]:
import yfinance as yf
import talib
import numpy as np 
import pandas as pd 
import vectorbt as vbt 
import warnings
from scipy import stats 
import matplotlib.pyplot as plt

In [11]:
#Download stock data from 2018 using yfinance

#Change these variable when needed
TICKER = "BTC-USD" #Ticker symbol (e.g. "QQQ", "SPY", "GLD")
START_DATE = "2018-01-01" #Start date (YYYY-MM-DD) format

#Download data from start date to today
stock_data = yf.download(TICKER, start=START_DATE, interval="1d")

#check if data was downloaded successfully (not empty)
if not stock_data.empty:
    print(f"Successfully downloaded {len(stock_data)} records for {TICKER} from {START_DATE} to today")
    print(f"Data range: {stock_data.index.min().date()} to {stock_data.index.max().date()}")
    print(stock_data.head())
else:
    print(f"Failed to download data for {TICKER} from yfinance")

#Display downloaded data
stock_data

[*********************100%***********************]  1 of 1 completed

Successfully downloaded 2921 records for BTC-USD from 2018-01-01 to today
Data range: 2018-01-01 to 2025-12-30
Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2018-01-01  13657.200195  14112.200195  13154.700195  14112.200195   
2018-01-02  14982.099609  15444.599609  13163.599609  13625.000000   
2018-01-03  15201.000000  15572.799805  14844.500000  14978.200195   
2018-01-04  15599.200195  15739.700195  14522.200195  15270.700195   
2018-01-05  17429.500000  17705.199219  15202.799805  15477.200195   

Price            Volume  
Ticker          BTC-USD  
Date                     
2018-01-01  10291200000  
2018-01-02  16846600192  
2018-01-03  16871900160  
2018-01-04  21783199744  
2018-01-05  23840899072  


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2018-01-01,13657.200195,14112.200195,13154.700195,14112.200195,10291200000
2018-01-02,14982.099609,15444.599609,13163.599609,13625.000000,16846600192
2018-01-03,15201.000000,15572.799805,14844.500000,14978.200195,16871900160
2018-01-04,15599.200195,15739.700195,14522.200195,15270.700195,21783199744
2018-01-05,17429.500000,17705.199219,15202.799805,15477.200195,23840899072
...,...,...,...,...,...
2025-12-26,87301.429688,89459.429688,86628.140625,87235.507812,42455674908
2025-12-27,87802.156250,87874.781250,87182.976562,87301.429688,13741199310


In [12]:
#Technical analysis indicators using talib

#Make sure stock_data is available from previous cell
if "stock_data" not in locals():
    raise ValueError("stock_data is not defined. Please run the previous cell first.")

#EXTRACT OHLCV data 
# yfinance returns either:
# 1) MultiIndex columns when multiple tickers are downloaded, e.g. ("Close", "QQQ")
# 2) Single-level columns when a single ticker is downloaded, e.g. "Close"
#
# This block handles both cases and extracts OHLCV data
# as NumPy arrays (open_, high, low, close, volume),
# which are required inputs for TA-Lib indicator functions.

if isinstance(stock_data.columns, pd.MultiIndex):
    # MultiIndex case
    open_ = stock_data["Open", TICKER].values
    high = stock_data["High", TICKER].values
    low = stock_data["Low", TICKER].values
    close = stock_data["Close", TICKER].values
    volume = stock_data["Volume", TICKER].values
else:
    # Single-level case
    open_ = stock_data["Open"].values
    high = stock_data["High"].values
    low = stock_data["Low"].values
    close = stock_data["Close"].values
    volume = stock_data["Volume"].values

    print(f"Calculating technical indicators using TA-Lib for {TICKER} from {START_DATE} to today")

#first calculate simple moving averages (SMA)
#defines the trend
sma_20 = talib.SMA(close, timeperiod=20)
sma_50 = talib.SMA(close, timeperiod=50)

#Exponential moving averages (EMA)
#good for signal timing, time the trend
ema_12 = talib.EMA(close, timeperiod=12)
ema_26 = talib.EMA(close, timeperiod=26)

#Moving average convergence divergence (MACD) 
#shows whether momentum is increasing or decreasing by comparing fast and slow trends
macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

#RSI (Relative Strength Index)
#RSI shows whether recent momentum is stretched, not whether price is cheap or expensive
rsi = talib.RSI(close, timeperiod=14)

#Stochastic RSI
#shows how extreme RSI is relative to its recent range, making it a fast timing tool—not a trend indicator
stochrsi_k, stochrsi_d = talib.STOCHRSI(close, timeperiod=14, fastk_period=3, fastd_period=3, fastd_matype=0)

#VWAP (Volume Weighted Average Price)
#the volume-weighted average price up to the current point
#where high-volume trades influence the average more than low-volume trades.
typical_price = (high + low + close) / 3
price_volume = typical_price * volume
cumulative_price_volume = np.cumsum(price_volume)
cumulative_volume = np.cumsum(volume)
vwap = cumulative_price_volume / cumulative_volume

#schaff trend cycle (STC)
#STC turns MACD into a fast, normalized momentum cycle that highlights trend accelerations and slowdowns earlier than MACD alone
cycle_period = 10

# Smooth MACD to reduce noise
macd_cycle = talib.EMA(macd, timeperiod=cycle_period)
macd_smooth = talib.EMA(macd_cycle, timeperiod=cycle_period)

# Find recent high and low of the smoothed MACD
highest_macd = talib.MAX(macd_smooth, timeperiod=cycle_period)
lowest_macd = talib.MIN(macd_smooth, timeperiod=cycle_period)

# Scale MACD into a 0–100 range 
stc_k = 100 * (macd_smooth - lowest_macd) / (highest_macd - lowest_macd) #stochastic k

# Smooth the cycle for cleaner signals
stc_d = talib.EMA(stc_k, timeperiod=3) 

# Unpack Stochastic RSI (returns fastk and fastd)

indicators_df = pd.DataFrame({
    "Date": stock_data.index,
    "Close": close,
    "SMA_20": sma_20,
    "SMA_50": sma_50,
    "EMA_12": ema_12,
    "EMA_26": ema_26,
    "MACD": macd,
    "MACD_Signal": macd_signal,
    "MACD_Hist": macd_hist,
    "RSI": rsi,
    "StochRSI_K": stochrsi_k,
    "StochRSI_D": stochrsi_d,
    "VWAP": vwap,
    "STC_K": stc_k,
    "STC_D": stc_d
})

print("All technical indicators calculated!")
print(f"Data shape: {indicators_df.shape}")
indicators_df.tail(5)

All technical indicators calculated!
Data shape: (2921, 15)


,Date,Close,SMA_20,SMA_50,EMA_12,EMA_26,MACD,MACD_Signal,MACD_Hist,RSI,StochRSI_K,StochRSI_D,VWAP,STC_K,STC_D
2916,2025-12-26,87301.429688,88800.272656,91442.518750,87933.014238,89246.483389,-1313.469151,-1490.742223,177.273071,42.642249,25.163033,16.617129,50816.519866,100.0,99.999986
2917,2025-12-27,87802.156250,88670.098437,91131.113750,87912.882239,89139.496194,-1226.613954,-1437.916569,211.302615,44.616331,100.000000,41.721011,50822.256556,100.0,99.999993
2918,2025-12-28,87835.835938,88529.880078,90842.188125,87901.028962,89042.928767,-1141.899805,-1378.713216,236.813411,44.754060,100.000000,75.054344,50828.602557,100.0,99.999996
2919,2025-12-29,87138.140625,88252.201563,90490.558125,87783.661526,88901.833349,-1118.171823,-1326.604938,208.433114,42.401671,0.000000,66.666667,50849.029233,100.0,99.999998
2920,2025-12-30,87956.203125,88048.964453,90129.750312,87810.206387,88831.786666,-1021.580279,-1265.600006,244.019727,45.986624,100.000000,66.666667,50864.306246,100.0,99.999999


In [13]:
#prepare price series 
#we need to split the data into training and testing data (in sample and out of sample)
#for robustness our ratio of IS to OOS will be 60/40 or 70/30
#suppress warnings from appearing in the output
warnings.filterwarnings("ignore", message="Degrees of freedom <= 0 for slice", category=RuntimeWarning)
warnings.filterwarnings("ignore", message="invalid value encountered in scalar divide", category=RuntimeWarning)

# Function to extract the 'Close' price series from a DataFrame
# This function handles different DataFrame column structures (MultiIndex vs single-level)
# Expect stock_data and TICKER already exist from previous cells
def select_close_series(df, ticker):
    """
    Extracts the 'Close' price column from a DataFrame, handling both MultiIndex and single-level columns.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The stock data DataFrame (from yfinance)
    ticker : str
        The ticker symbol (e.g., "BTC-USD", "QQQ")
    
    Returns:
    --------
    pandas.Series
        The Close price series as a float Series
    """
    # Check if DataFrame has MultiIndex columns (happens when downloading multiple tickers)
    if isinstance(df.columns, pd.MultiIndex):
        # Try to select the Close column for the specific ticker
        if ("Close", ticker) in df.columns:
            s = df[("Close", ticker)]
        else:
            # If ticker-specific column not found, search for any column containing 'Close'
            cols = [c for c in df.columns if 'Close' in str(c)]
            if not cols:
                raise KeyError("Close not found")
            s = df[cols[0]]
    else:
        # Single-level columns case - search for any column containing 'Close'
        cols = [c for c in df.columns if 'Close' in str(c)]
        if not cols:
            raise KeyError("Close not found")
        s = df[cols[0]]
    
    # Convert to float and squeeze to ensure it's a 1D Series (remove any extra dimensions)
    return s.astype(float).squeeze()

# Extract the Close price series using our function
# This gives us a clean pandas Series with just the closing prices
close = select_close_series(stock_data, TICKER)

# Rename the series to 'price' for clarity in subsequent analysis
close.name = 'price'

# Simple split: Divide data into training and validation sets
# TRAIN_RATIO determines what percentage of data is used for training
# 0.6 means 60% training, 40% validation
TRAIN_RATIO = 0.6

# Calculate the index where we'll split the data
# int() truncates to ensure we get a valid integer index
split_idx = int(len(close) * TRAIN_RATIO)

# Create training set: all data from the beginning up to split_idx
# .copy() ensures we get an independent copy, not a view
train_close = close.iloc[:split_idx].copy()

# Create validation set: all data from split_idx to the end
# This is our out-of-sample data for testing model performance
val_close = close.iloc[split_idx:].copy()

# Print the date ranges for both datasets to verify the split
# This helps us understand what time periods we're training and validating on
print(f"Data ready: train={train_close.index[0].date()} → {train_close.index[-1].date()} | val={val_close.index[0].date()} → {val_close.index[-1].date()}")

Data ready: train=2018-01-01 → 2022-10-18 | val=2022-10-19 → 2025-12-30
